In [12]:
##another update--final

In [1]:
import os
import cv2
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation
from tkinter import filedialog, Tk, Button
import numpy as np
import threading
import queue

# Globals
imgList = []
current_bg_index = 0
remove_whole_bg = False
running = True

# Queues to safely trigger file dialog from the main thread
upload_request_queue = queue.Queue()

# Initialize camera and segmentation
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
segmentor = SelfiSegmentation()

# --- Triggered from GUI thread: Request upload ---
def upload_background_request():
    upload_request_queue.put(True)

# --- Remove Background to White ---
def set_remove_background():
    global remove_whole_bg
    remove_whole_bg = True

# --- Cancel Program ---
def cancel_program():
    global running
    running = False
    try:
        gui.destroy()
    except:
        pass

# --- GUI thread ---
def start_gui():
    global gui
    gui = Tk()
    gui.title("Background Controller")
    gui.geometry("220x150")

    Button(gui, text="Upload Background", command=upload_background_request).pack(pady=5)
    Button(gui, text="Remove Background", command=set_remove_background).pack(pady=5)
    Button(gui, text="Cancel Program", command=cancel_program).pack(pady=5)

    gui.mainloop()

# GUI thread start
gui_thread = threading.Thread(target=start_gui)
gui_thread.daemon = True
gui_thread.start()

# Default black background
imgList.append(np.zeros((480, 640, 3), dtype=np.uint8))

# Main camera loop
while running:
    success, img = cap.read()
    if not success:
        print("Camera read failed.")
        break

    # Handle upload requests
    if not upload_request_queue.empty():
        upload_request_queue.get()
        # Temporarily create a hidden root window for file dialog
        temp_root = Tk()
        temp_root.withdraw()
        file_path = filedialog.askopenfilename(
            title="Select a background image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp")]
        )
        temp_root.destroy()
        if file_path:
            new_bg = cv2.imread(file_path)
            if new_bg is not None:
                new_bg = cv2.resize(new_bg, (640, 480))
                imgList.append(new_bg)
                current_bg_index = len(imgList) - 1
                remove_whole_bg = False  # Reset background flag
            else:
                print("Error: Failed to load image")

    # Choose background
    if remove_whole_bg:
        blank = np.ones((480, 640, 3), dtype=np.uint8) * 255  # white background
        imgOut = segmentor.removeBG(img, blank, cutThreshold=0.8)
    else:
        imgBg = imgList[current_bg_index]
        imgOut = segmentor.removeBG(img, imgBg, cutThreshold=0.8)

    stackedImg = cvzone.stackImages([img, imgOut], 2, 1)
    cv2.imshow("Live Segmentation", stackedImg)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        current_bg_index = (current_bg_index - 1) % len(imgList)
        remove_whole_bg = False
    elif key == ord('d'):
        current_bg_index = (current_bg_index + 1) % len(imgList)
        remove_whole_bg = False
    elif key == ord('q'):
        running = False
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
